In [1]:
import pandas as pd
import numpy as np
import nltk
from collections import Counter
from nltk.corpus import stopwords
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy.optimize import minimize
stops = set(stopwords.words("english"))
import xgboost as xgb
from sklearn.cross_validation import train_test_split
import multiprocessing
import difflib

/home/takaya/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
features = [
#     'clean_q1_tokenized',
#     'clean_q2_tokenized',
#     'clean_q1_stem',
#     'clean_q2_stem',
#     'clean_q1_pos_tagged',
#     'clean_q2_pos_tagged',
    'clean_q1_lemmatized',
    'clean_q2_lemmatized',
    'clean_q1_lemmatized_stem',
    'clean_q2_lemmatized_stem'
]

In [4]:
def read_data(t, features):
    data = pd.read_csv('../input/{}.csv'.format(t))
    for feature in features:
        data = pd.merge(data, pd.read_pickle('../feature/{}_{}.pkl'.format(t, feature)))
    data.fillna(0.0)
    return data

In [62]:
#train = read_data('train', features)#[:1000]
#test = read_data('test', features)#[:1000]

In [5]:
train = pd.read_pickle('train_with_pred.pkl')

In [17]:
train = train[:5000]

In [28]:
bad_data = train.sort_values(by='loss', ascending=False)[:5000]

In [26]:
analysis_features = [
    'id',
    'question1',
    'question2',
    'is_duplicate',
    'pred',
    'loss',
#     'clean_q1_lemmatized',
#     'clean_q2_lemmatized',
#     'q1_lemmatized',
#     'q2_lemmatized',
    'clean_q1_lemmatized_stem',
    'clean_q2_lemmatized_stem',
#     'question1_nouns',
#     'question2_nouns',
    
    'sum_prob_weight_common_words',
    'z_match_ratio',
    'n_sim_lemm',
    
    'bleu'
]

In [ ]:
from multiprocessing import Pool

In [ ]:
def parallel(df, func):
    splitted_df = np.array_split(df, 32)
    pool = Pool(6)
    df = pd.concat(pool.map(func, splitted_df))
    pool.close()
    pool.join()
    return df

In [ ]:
def bleu(a, b):
    return nltk.translate.bleu_score.sentence_bleu([a], b)

def wmd(df):
    df['clean_lemm_wmd'] = df.apply(gao, axis=1)
    return df

In [19]:
import en_core_web_md
nlp = en_core_web_md.load()

In [20]:
def get_span(s):
    return list(nlp(s).sents)[0]

In [21]:
def show_info(s):
    sent = get_span(s)
    for node in sent:
        if node.head == sent.root:
            print('{:9}: {}'.format(str(node), ' '.join(map(str, node.subtree))))

In [30]:
def show_q1q2(row):
    print('loss: {}'.format(row.loss))
    print('pred: {}'.format(row.pred))
    print('is_duplicate: {}'.format(row.is_duplicate))
    print(row.question1)
    print(row.question2)
    print('*' * 70)
    
    show_info(row.question1)
    print('-' * 60)
    show_info(row.question2)
    
    print('*' * 70)
    print('\n')
bad_data[1000:1500].apply(show_q1q2, axis=1)

loss: 2.6438664827370477
pred: 0.07108588516712189
is_duplicate: 1
What is clearance volume in engine?
What is clearance volume?
**********************************************************************
What     : What
is       : What is clearance volume in engine ?
volume   : clearance volume in engine
?        : ?
------------------------------------------------------------
What     : What
is       : What is clearance volume ?
volume   : clearance volume
?        : ?
**********************************************************************


loss: 2.6432859984753416
pred: 0.07112716138362885
is_duplicate: 1
What is borderline personality disorder?
What is a borderline personality?
**********************************************************************
What     : What
is       : What is borderline personality disorder ?
disorder : borderline personality disorder
?        : ?
------------------------------------------------------------
What     : What
is       : What is a borderline personali

96487     None
14996     None
235242    None
73873     None
245586    None
29818     None
289448    None
220980    None
234183    None
140463    None
165058    None
211726    None
369444    None
2166      None
19020     None
220707    None
258435    None
186216    None
188040    None
155917    None
115311    None
286398    None
76698     None
216289    None
313908    None
329855    None
251487    None
213910    None
102036    None
163803    None
          ... 
387466    None
287789    None
287065    None
149065    None
251243    None
80438     None
245500    None
312402    None
287426    None
394397    None
249031    None
390307    None
167801    None
352755    None
349098    None
18117     None
286322    None
312961    None
228170    None
320158    None
105209    None
307985    None
250330    None
259998    None
387930    None
246602    None
216727    None
147242    None
193848    None
77065     None
dtype: object

In [31]:
q = get_span('I am a programmer')

In [ ]:
train = parallel(train, wmd)

In [ ]:
test = parallel(test, wmd)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

plt.rcParams['figure.figsize'] = (15.0, 15.0)

In [ ]:
plt.hist(train.clean_lemm_wmd[train['is_duplicate'] == 0], bins=20, normed=True, label='0')
plt.hist(train.clean_lemm_wmd[train['is_duplicate'] == 1], bins=20, normed=True, label='1', alpha=0.7)
None

In [ ]:
print(',\n'.join(map(lambda s: "'{}'".format(s), (col for col in train.columns if col.startswith('clean_')))))

In [ ]:
features_to_save = [
    'clean_lemm_wmd',
]

In [ ]:
train

In [ ]:
import util
util.save_feature(train, 'train', features_to_save, 'id')
util.save_feature(test, 'test', features_to_save, 'test_id')